In [4]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats

In [5]:
#generate the sample
def gen_data():
    nobs = 1000
    beta = 3
    x = np.random.uniform(low=-3., high=3., size=nobs)
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1*(1 + beta * x + e >= 0) 
    return y,x,nobs

In [15]:
def ndVuong(model1,model2,alpha,rs,S_cv):
    
    model1_fit = model1.fit(disp=False)
    ll1 = model1.loglikeobs(model1_fit.params)
    grad1 =  model1.score(model1_fit.params)
    hess1 =  model1.hessian(model1_fit.params)
    print(hess1.shape)
    
    model2_fit = model2.fit(disp=False)
    ll2 = model2.loglikeobs(model2_fit.params)
    grad2 =  model2.score(model2_fit.params)
    hess2 =  model2.hessian(model2_fit.params)
    
    k1 = len(model1_fit.params)
    k2 = len(model2_fit.params)
    k = k1 + k2
    n = len(ll1)
    
    #A_hat:
    A_hat1 = np.concatenate([hess1,np.zeros((k1,k2))])
    A_hat2 = np.concatenate([np.zeros((k2,k1)),-1*hess2])
    A_hat = np.concatenate([A_hat1,A_hat2],axis=1)
    
    #B_hat, covariance of the score...
    B_hat =  np.cov([grad1,grad2]) #might be a mistake here..
    

    #W_hat = (sqrtm(B_hat)/A_hat)*sqrtm(B_hat);
    #[~,V]=eig(W_hat);
    
    
    

yn,xn,nobs = gen_data()
model1 = sm.Probit(yn,sm.add_constant(xn))
model2 = sm.Logit(yn,sm.add_constant(xn))

print(ndVuong(model1,model2,.05,None,None))

(2, 2)
[[3.87069548e-29 8.79148501e-30]
 [8.79148501e-30 1.99680417e-30]]
None


In [ ]:
   
    #B_hat:
    meandL = mean([d_logf,-d_logg]);
    dmeandL = [d_logf,-d_logg] - repmat(meandL,n,1);
    B_hat =

    W_hat = (sqrtm(B_hat)/A_hat)*sqrtm(B_hat);
    [~,V]=eig(W_hat);


    vecV = diag(V);
    abs_vecV = abs(vecV)-max(abs(vecV));
    rho_star = 1*(abs_vecV==0);
    rnorm = rho_star'*rho_star;
    rho_star = sqrt(rnorm)^(-1)*rho_star;

    Z0 = randn(rs,S_cv,k+1);

    VZ = [1,rho_star';rho_star,eye(k)];

    Z = Z0*sqrtm(VZ+10^(-12)*eye(k+1));

    Z_L = Z(:,1);            #$Z_Lambda$
    Z_p = Z(:,2:k+1);        #$Z_phi^\ast$

    #trace(V)  #diagonostic line

    trVsq = trace(V^2);
    Vnmlzd = V/sqrt(trVsq);   #V, normalized by sqrt(trVsq);

    J_Lmod = @(sig,c)sig*Z_L - (Z_p.^2)*diag(Vnmlzd)/2+ trace(Vnmlzd)/2;

    J_omod = @(sig,c)sig^2 - 2*sig*Z_p*Vnmlzd*rho_star + ...
                                           (Z_p.^2)*(diag(Vnmlzd).^2) + c;

    quant = @(sig,c)quantile(abs(J_Lmod(sig,c)./sqrt(max(1e-10,J_omod(sig,c)))),1-alpha);

                                      # quantile as a function of sigma and c.
                                      # the lower bound 1e-10 is used so that
                                      # inf does not appear as a value of
                                      # quant; doing so ensures stability of
                                      # the solution.

    sigstar = @(c)fminbnd(@(sig)-quant(sig,c),0,5);

    cv0 = quant(sigstar(0),0);        # critical value with c=0

    z_normal = norminv(1-alpha/2);
    z_nor_sim = max(z_normal,quantile(abs(Z_L),1-alpha)); #simulated z_normal

    if cv0 - z_nor_sim <= 0.1;  # if critical value with c=0 is not very big

        cv = max(cv0,z_normal);                     # use c = 0

        cstar = 0;

    else                              # otherwise, increase c.

        Func = @(c)((quant(sigstar(c),c)-z_nor_sim)-0.1)^2;
        cstar = fminbnd(Func,0,10);
        cv = max(quant(sigstar(cstar),cstar),z_normal);
    end

    #Computing the ND test statistic:
    nLR_hat = sum(logf-logg);         #sample log-likelihood ratio times n

    nomega2_hat = (logf-logg)'*(logf-logg) - nLR_hat^2/n;
                                          #n times sample variance of \Lambda_i    
    #Non-degenerate Vuong Tests    
    Tnd = (nLR_hat+trace(V)/2)/sqrt(nomega2_hat + cstar*trace(V.^2));